In [68]:
from esper.prelude import *

def get_fps_map(vids):
    from query.models import Video
    vs = Video.objects.filter(id__in=vids)
    return {v.id: v.fps for v in vs}

def frame_second_conversion(c, mode='f2s'):
    from rekall.video_interval_collection_3d import VideoIntervalCollection3D
    from rekall.interval_set_3d import Interval3D
    fps_map = get_fps_map(set(c.get_allintervals().keys()))
    
    def second_to_frame(fps):
        def map_fn(intrvl):
            i2 = intrvl.copy()
            t1,t2 = intrvl.t
            i2.t = (int(t1*fps), int(t2*fps))
            return i2
        return map_fn
    
    def frame_to_second(fps):
        def map_fn(intrvl):
            i2 = intrvl.copy()
            t1,t2 = intrvl.t
            i2.t = (int(t1/fps), int(t2/fps))
            return i2
        return map_fn
    
    if mode=='f2s':
        fn = frame_to_second
    if mode=='s2f':
        fn = second_to_frame
    output = {}
    for vid, intervals in c.get_allintervals().items():
        output[vid] = intervals.map(fn(fps_map[vid]))
    return VideoIntervalCollection3D(output)

def frame_to_second_collection(c):
    return frame_second_conversion(c, 'f2s')

def second_to_frame_collection(c):
    return frame_second_conversion(c, 's2f')

def convert_to_1d_collection(collection):
    from rekall.interval_list import Interval
    from rekall.video_interval_collection import VideoIntervalCollection
    video_map = collection.get_allintervals()
    return VideoIntervalCollection({vid: [Interval(
        i.t[0], i.t[1], None) for i in video_map[vid].get_intervals()] for vid in video_map})

def display_result(collection_1d):
    from esper.rekall import intrvllists_to_result
    results = intrvllists_to_result(collection_1d.get_allintervals())
    return esper_widget(results,
            crop_bboxes=False, show_middle_frame=False, disable_captions=False,
            results_per_page=25, jupyter_keybindings=True)

# time dimension in seconds
def get_commercial_intervals_in_vids(vids, in_seconds=True):
    from query.models import Commercial
    from rekall.video_interval_collection_3d import VideoIntervalCollection3D
    
    qs = Commercial.objects.filter(video_id__in=vids)
    commercials = VideoIntervalCollection3D.from_django_qs(qs)
    if in_seconds:
        return frame_to_second_collection(commercials)
    return commercials

# Interviews

In [54]:
GUEST_LIST = [name.lower() for name in ['Barack Obama', 'Donald Trump', 'Ted Cruz', 'John Kasich', 'Marco Rubio', 'Ben Carson', 'Jeb Bush',
'Jim Gilmore', 'Chris Christie', 'Carly Fiorina', 'Rick Santorum', 'Rand Paul', 'Mike Huckabee',
'Hillary Clinton', 'Bernie Sanders', 'Lincoln Chafee', 'Martin O’Malley', 'Jim Webb',
'Sarah Palin', 'John Boehner', 'Paul Ryan', 'Newt Gingrich','Nancy Pelosi','Elizabeth Warren', 'Mitch McConnell',
'Chuck Schumer','Harry Reid','Joe Biden', 'Kevin McCarthy', 'Steve Scalise', 'Bobby Jindal', 'John Cornyn',
'Dick Durbin','Orrin Hatch', 'Lindsey Graham', 'Mitt Romney', 'Michelle Obama' ,'Bill Clinton', 
'George W Bush', 'Tim Kaine' ]]
HOST_LIST = list(set([h.name for s in CanonicalShow.objects.exclude(hosts=None) for h in s.hosts.all()]))
VIDEOS = sorted([v.id for v in Video.objects.exclude(show__hosts=None)])

def get_name_to_labeler_id(names):
    from tqdm import tqdm
    def get_labeler_ids(n):
        from query.models import Labeler
        labeler_names = ['face-identity:'+n, 'face-identity-converted:'+n, 'face-identity-uncommon:'+n]
        return [l.id for l in Labeler.objects.filter(name__in=labeler_names)]
    output = {}
    for n in tqdm(names):
        output[n] = get_labeler_ids(n)
    return output

NAME_TO_LABELER_ID = get_name_to_labeler_id(GUEST_LIST+HOST_LIST)

def name_to_id(name):
    from query.models import Identity
    return Identity.objects.get(name=name).id

GUEST_IDS=[name_to_id(n) for n in GUEST_LIST]
HOST_IDS=[name_to_id(n) for n in HOST_LIST]

# time dimension in seconds
# Outputs a dictionary from name to video interval collection
def get_person_intervals_in_vids(person_names, vids, probability=0.7, min_height=None):
    from query.models import FaceIdentity
    from django.db.models import F,Q
    from rekall.video_interval_collection_3d import VideoIntervalCollection3D
    from rekall.interval_set_3d import Interval3D
    from rekall.interval_set_3d_utils import P
    
    SAMPLE_RATE = 3 # Every 3s

    lids = []
    for n in person_names:
        lids.extend(NAME_TO_LABELER_ID[n])
    
    face_id_qs = FaceIdentity.objects.filter(
        probability__gte=probability,
        face__frame__video_id__in=vids,
        face__frame__shot_boundary=False,
        labeler_id__in=lids,
    ).annotate(
        height=F('face__bbox_y2')-F('face__bbox_y1'),
        labeler_name=F('labeler__name'),
        video_id=F('face__frame__video_id'),
        frame_number=F('face__frame__number'),
        x1=F('face__bbox_x1'),
        x2=F('face__bbox_x2'),
        y1=F('face__bbox_y1'),
        y2=F('face__bbox_y2'),
    )
    if min_height is not None:
        face_id_qs = face_id_qs.filter(height__gte=min_height)
    
    total = face_id_qs.count()
    faces = VideoIntervalCollection3D.from_django_qs(face_id_qs, {
        't1':'frame_number',
        't2':'frame_number',
        'x1':'x1','x2':'x2','y1':'y1','y2':'y2',
    }, with_payload=lambda row: row.labeler_name.split(':')[1], progress=False, total=total)

    fps_map = get_fps_map(set(faces.get_allintervals().keys()))
    names_to_collection = {}
    for n in person_names:
        faces_one_person = faces.filter(P(lambda p: p==n))
        output = {}
        for vid, intervals in faces_one_person.get_allintervals().items():
            fps = fps_map[vid]
            eps = round(fps * SAMPLE_RATE)
            output[vid] = intervals.temporal_coalesce(epsilon=eps)
        names_to_collection[n] = frame_to_second_collection(VideoIntervalCollection3D(output))
    return names_to_collection

# Returns interview_IS<person_only_IS<>, host_only_IS<>, person_with_host_IS<>>
def interview_query(guest, hosts, commercials):
    from rekall.interval_set_3d import Interval3D
    from rekall.interval_set_3d_utils import T, P, or_preds, overlap_bound
    from rekall.temporal_predicates import overlaps, before, after
    
    SEGMENT_LENGTH=30
    OVERLAP_LAX=60
    HOST_GUEST_GAP=120
    MIN_LENGTH=240
    SMALL_FACE_THRESHOLD=0.3
    MIN_GUEST_TIME_RATIO=0.35
    MAX_SMALL_GUEST_RATIO=0.7
    
    fuzzy_overlap = or_preds(overlaps(), before(max_dist=OVERLAP_LAX), after(max_dist=OVERLAP_LAX))
    
    interview_candidates = hosts.merge(guest, T(fuzzy_overlap), time_window=OVERLAP_LAX).temporal_coalesce()
    interviews = interview_candidates.temporal_coalesce(
        epsilon=HOST_GUEST_GAP
    ).filter_size(min_size=MIN_LENGTH
    ).minus(commercials
    ).filter_size(min_size=MIN_LENGTH)

    def select_second(p):
        return p[1]
    
    # Interview<Guest<height>>
    interview_with_guest = interviews.collect_by_interval(
        guest,
        T(overlaps()),
        filter_empty=True,
        time_window=0,
    ).map_payload(
        select_second)

    def total_time(intervals):
        return intervals.fold(lambda s, i: s+i.length(), 0)
    
    def filter_time(interview):
        guest = interview.payload
        small_guest = guest.filter_size(max_size=SMALL_FACE_THRESHOLD, axis='Y')
        small_guest_time = total_time(small_guest)
        total_guest_time = total_time(guest)
        segment_time = interview.length()
        return (total_guest_time / segment_time > MIN_GUEST_TIME_RATIO and
                small_guest_time / total_guest_time < MAX_SMALL_GUEST_RATIO)
    # Interview<Guest<height>>
    interviews = interview_with_guest.filter(filter_time)

    # Guest<height>
    guest_in_interviews = guest.filter_against(interviews, T(overlaps()), time_window=0)
    # HostAndGuest<(Host, Guest)>
    guest_with_host = guest_in_interviews.join(
        hosts,
        T(overlaps()),
        lambda guest, host: [Interval3D(overlap_bound(guest.t, host.t), payload=(guest, host))],
        time_window=0)
    guest_only = guest_in_interviews.minus(guest_with_host)

    hosts_in_interviews = hosts.filter_against(interviews, T(overlaps()), time_window=0)
    hosts_only = hosts_in_interviews.minus(guest_with_host)
    
    interview_with_metadata = interviews.collect_by_interval(
        guest_only,
        T(overlaps()),
        filter_empty=False,
        time_window=0
    ).map_payload(select_second).collect_by_interval(
        hosts_only,
        T(overlaps()),
        filter_empty=False,
        time_window=0
    ).collect_by_interval(
        guest_with_host,
        T(overlaps()),
        filter_empty=False,
        time_window=0
    ).map_payload(lambda p: (p[0][0],p[0][1],p[1]))
    
    return interview_with_metadata

def get_interviews_for_vids(vids):
    from rekall.video_interval_collection_3d import VideoIntervalCollection3D
    from tqdm import tqdm
    
    people_to_intervals = get_person_intervals_in_vids(HOST_LIST + GUEST_LIST, vids, 0.7,0.2)
    hosts = VideoIntervalCollection3D({})
    for host_name in HOST_LIST:
        hosts = hosts.union(people_to_intervals[host_name])
    commercials = get_commercial_intervals_in_vids(vids)
    ret = VideoIntervalCollection3D({})
    for guest_name in tqdm(GUEST_LIST):
        guest = people_to_intervals[guest_name]
        interviews = interview_query(guest, hosts, commercials)
        ret = ret.union(interviews)
    return ret       

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 211/211 [00:00<00:00, 637.86it/s]


## Run on a few videos

In [57]:
vids = VIDEOS[::10000]
answer = get_interviews_for_vids(vids)
display_result(convert_to_1d_collection(second_to_frame_collection(answer)))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:00<00:00, 230.79it/s]


VGridWidget(jsglobals={'schema': [['IdentityTag', ['id', 'name']], ['Identity', ['id', 'name', 'tags']], ['Can…

## Run On All of TVNews

In [46]:
import ipyparallel as ipp
from esper.rekall_parallel import get_runtime_for_ipython_cluster
import pickle
c = ipp.Client(profile='local')
rt = get_runtime_for_ipython_cluster(c)

vids = VIDEOS
answer,_ = rt.run(get_interviews_for_vids, vids, randomize=False, chunksize=20, progress=True)
pickle.dump(answer, open('../data/interviews/interviews{0}-{1}.pkl'.format(vids[0],vids[-1]), 'wb'))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 195445/195445 [43:59<00:00, 85.06it/s]


# Faces in a Row

In [120]:
VIDEOS = sorted([v.id for v in Video.objects.all()])

def get_faces_in_a_row_for_vids(vids):
    from rekall.video_interval_collection_3d import VideoIntervalCollection3D
    from rekall.interval_set_3d_utils import P, XY, and_preds
    from rekall.bbox_predicates import left_of, same_value
    
    MIN_NUM_FACES = 7
    MIN_HEIGHT = 0.15
    EPSILON = 0.1
    
    qs = Face.objects.filter(frame__video_id__in=vids).annotate(
        video_id=F("frame__video_id"),
        min_frame=F("frame__number"),
        max_frame=F("frame__number") + 1,
        height=F('bbox_y2')-F('bbox_y1'),
    ).filter(height__gte=MIN_HEIGHT)
    total = qs.count()
    faces = VideoIntervalCollection3D.from_django_qs(qs, VideoIntervalCollection3D.django_bbox_default_schema(),
                                            progress=True, total=total)
    def has_enough_faces(n):
        def pred(faces):
            return faces.size() >= n
        return pred
    
    def get_pattern(n):
        assert(n>1)
        constraints = []
        for i in range(n-1):
            name1 = str(i)
            name2 = str(i+1)
            constraints.append(([name1, name2],[XY(
                and_preds(
                    left_of(),
                    same_value('y1', epsilon=EPSILON),
                    same_value('y2', epsilon=EPSILON)))]))
        return constraints          
    
    def faces_aligned():
        def pred(faces):
            pattern = get_pattern(faces.size())
            return len(faces.match(pattern, exact=True)) > 0
        return pred
        
    commercials = get_commercial_intervals_in_vids(vids, in_seconds=False)
    
    aligned_faces_in_frames = faces.minus(commercials).group_by_time().filter(P(and_preds(
        has_enough_faces(MIN_NUM_FACES),
        faces_aligned())))
    
    return aligned_faces_in_frames

## Run on a few videos

In [121]:
vids = VIDEOS[::10003]
answer = get_faces_in_a_row_for_vids(vids)
display_result(convert_to_1d_collection(answer))

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 32607/32607 [00:00<00:00, 33943.44it/s]


VGridWidget(jsglobals={'schema': [['IdentityTag', ['id', 'name']], ['Identity', ['id', 'name', 'tags']], ['Can…

## Run On All of TVNews

In [122]:
import ipyparallel as ipp
from esper.rekall_parallel import get_runtime_for_ipython_cluster
import pickle
c = ipp.Client(profile='local')
rt = get_runtime_for_ipython_cluster(c)

vids = VIDEOS[2::10]
answer,_ = rt.run(get_faces_in_a_row_for_vids, vids, randomize=False, chunksize=15, progress=True)
print("Total results:", sum(answer.size().values()))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 21489/21489 [03:42<00:00, 176.99it/s]

Total results: 164


In [123]:
def filter_vids(c, vids=None):
    from rekall.video_interval_collection_3d import VideoIntervalCollection3D
    if vids is None or len(vids)==0:
        return c
    d = c.get_allintervals()
    ret = {}
    for v in vids:
        if v in d:
            ret[v] = d[v]
    return VideoIntervalCollection3D(ret)

display_result(convert_to_1d_collection(filter_vids(answer,[])))

VGridWidget(jsglobals={'schema': [['IdentityTag', ['id', 'name']], ['Identity', ['id', 'name', 'tags']], ['Can…

# Scratchpad

In [5]:
vids = [763, 3769, 5281, 8220, 9901, 12837, 13141, 26386, 33004, 33004, 34642, 38275, 42756, 50164, 50164, 50164, 50164, 50164, 50164, 50164, 52075, 52945, 54377, 54377, 59122, 59122, 59398, 59398, 59398, 59398]

answer = get_person_intervals_in_vids(HOST_LIST + GUEST_LIST, vids, 0.7,0.2)
display_result(convert_to_1d_collection(second_to_frame_collection(answer['bernie sanders'])))

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 11590/11590 [00:00<00:00, 21759.72it/s]


VGridWidget(jsglobals={'schema': [['IdentityTag', ['id', 'name']], ['Identity', ['id', 'name', 'tags']], ['Can…

In [ ]:
display_result(convert_to_1d_collection(second_to_frame_collection(answer['jake tapper'])))

In [ ]:
answer = get_interviews_for_vids(vids)

In [ ]:
display_result(convert_to_1d_collection(second_to_frame_collection(answer)))

In [ ]:
ls=[l.name for l in Labeler.objects.all() if l.name.startswith('face-identity:') or l.name.startswith('face-identity-converted:') or l.name.startswith('face-identity-uncommon:')]

In [ ]:
ls=[l.split(':')[1] for l in ls]

In [ ]:
for g in GUEST_LIST:
    if g not in ls:
        print(g)

In [ ]:
for h in HOST_LIST:
    if h not in ls:
        print(h)

In [ ]:
ls

In [ ]:
sorted(HOST_LIST)

In [ ]:
interviews = LabeledInterview.objects \
        .annotate(fps=F('video__fps')) \
        .annotate(min_frame=F('fps') * F('start')) \
        .annotate(max_frame=F('fps') * F('end')) \
        .filter(guest1="bernie sanders", original=True)
print([i.video.id for i in interviews])

In [ ]:
len(vids)

In [ ]:
answer.get_allintervals()[10]

In [ ]:
answer

In [ ]:
len(VIDEOS)

In [ ]:
VIDEOS[:100]

In [ ]:
Video.objects.count()

In [16]:
from rekall.video_interval_collection_3d import VideoIntervalCollection3D
vids = [188346]
people_to_intervals = get_person_intervals_in_vids(HOST_LIST + GUEST_LIST, vids, 0.7,0.2)
hosts = VideoIntervalCollection3D({})
for host_name in HOST_LIST:
    hosts = hosts.union(people_to_intervals[host_name])

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 495/495 [00:00<00:00, 18738.77it/s]


In [17]:
display_result(convert_to_1d_collection(second_to_frame_collection(hosts)))

VGridWidget(jsglobals={'schema': [['IdentityTag', ['id', 'name']], ['Identity', ['id', 'name', 'tags']], ['Can…

In [7]:
import pickle
a = pickle.load(open('../data/interviews/paper/interview_10y-all.pkl', 'rb'))

In [12]:
a['John Kasich'][188346]

{'all': [(2503.0020000000013,
   2974.503333333335,
   [(0.459918588399887, 210.24033333333364),
    (0.30950930714607294, 21.051333333333332),
    (0.30319644510746, 186.2163333333333),
    (0.299088642001152, 21.051333333333332)])],
 'host_only': [(2752.2510000000016,
   2764.2630000000017,
   [(0.459918588399887, 210.24033333333364),
    (0.30950930714607294, 21.051333333333332),
    (0.30319644510746, 186.2163333333333),
    (0.299088642001152, 21.051333333333332)]),
  (2950.479333333335,
   2953.4520000000016,
   [(0.459918588399887, 210.24033333333364),
    (0.30950930714607294, 21.051333333333332),
    (0.30319644510746, 186.2163333333333),
    (0.299088642001152, 21.051333333333332)])],
 'person_only': [(2503.0020000000013,
   2506.0050000000015,
   [(0.459918588399887, 210.24033333333364),
    (0.30950930714607294, 21.051333333333332),
    (0.30319644510746, 186.2163333333333),
    (0.299088642001152, 21.051333333333332)]),
  (2509.038333333335,
   2512.0110000000013,
   [(0.4

In [13]:
VIDEOS.index(188346)

173481

In [20]:
people_to_intervals['john kasich'].get_allintervals()[188346]

[<Interval3D t: [48,48] x: [0.0729675069451332,0.340209662914276] y: [0.192048326134682,0.7034050822258] payload:john kasich>, <Interval3D t: [51,51] x: [0.0788620859384537,0.33420005440712] y: [0.200570270419121,0.697366416454315] payload:john kasich>, <Interval3D t: [54,54] x: [0.0818389803171158,0.335762768983841] y: [0.197350889444351,0.691627025604248] payload:john kasich>, <Interval3D t: [2183,2183] x: [0.397857904434204,0.620155453681946] y: [0.129200682044029,0.578142821788788] payload:john kasich>, <Interval3D t: [2186,2186] x: [0.408488303422928,0.63443648815155] y: [0.134875744581223,0.592679738998413] payload:john kasich>, <Interval3D t: [2189,2189] x: [0.395539820194244,0.619785845279694] y: [0.134044304490089,0.582561612129211] payload:john kasich>, <Interval3D t: [2504,2504] x: [0.39989972114563,0.629268884658813] y: [0.105802565813065,0.565721154212952] payload:john kasich>, <Interval3D t: [2507,2507] x: [0.685153722763062,0.837928771972656] y: [0.168197274208069,0.4720

In [38]:
Video.objects.get(id=188346).fps * 3

179.8201798201797

In [37]:
def get_person_intervals_in_vids_frames(person_names, vids, probability=0.7, min_height=None):
    from query.models import FaceIdentity
    from django.db.models import F,Q
    from rekall.video_interval_collection_3d import VideoIntervalCollection3D
    from rekall.interval_set_3d import Interval3D
    from rekall.interval_set_3d_utils import P
    
    SAMPLE_RATE = 3 # Every 3s

    lids = []
    for n in person_names:
        lids.extend(NAME_TO_LABELER_ID[n])
    
    face_id_qs = FaceIdentity.objects.filter(
        probability__gte=probability,
        face__frame__video_id__in=vids,
        face__frame__shot_boundary=False,
        labeler_id__in=lids,
    ).annotate(
        height=F('face__bbox_y2')-F('face__bbox_y1'),
        labeler_name=F('labeler__name'),
        video_id=F('face__frame__video_id'),
        frame_number=F('face__frame__number'),
        x1=F('face__bbox_x1'),
        x2=F('face__bbox_x2'),
        y1=F('face__bbox_y1'),
        y2=F('face__bbox_y2'),
    )
    if min_height is not None:
        face_id_qs = face_id_qs.filter(height__gte=min_height)
    
    total = face_id_qs.count()
    faces = VideoIntervalCollection3D.from_django_qs(face_id_qs, {
        't1':'frame_number',
        't2':'frame_number',
        'x1':'x1','x2':'x2','y1':'y1','y2':'y2',
    }, with_payload=lambda row: row.labeler_name.split(':')[1], progress=True, total=total)

    fps_map = get_fps_map(set(faces.get_allintervals().keys()))
    names_to_collection = {}
    for n in person_names:
        faces_one_person = faces.filter(P(lambda p: p==n))
        output = {}
        for vid, intervals in faces_one_person.get_allintervals().items():
            fps = fps_map[vid]
            eps = fps * SAMPLE_RATE
            output[vid] = intervals.temporal_coalesce(epsilon=eps)
        names_to_collection[n] = VideoIntervalCollection3D(output)
    return names_to_collection
jk = get_person_intervals_in_vids_frames(['john kasich'], [188346], probability=0.7, min_height=0.2)
jk['john kasich'].get_allintervals()


100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 152/152 [00:00<00:00, 11377.02it/s]


{188346: [<Interval3D t: [2880,2880] x: [0.0729675069451332,0.340209662914276] y: [0.192048326134682,0.7034050822258] payload:john kasich>, <Interval3D t: [3060,3060] x: [0.0788620859384537,0.33420005440712] y: [0.200570270419121,0.697366416454315] payload:john kasich>, <Interval3D t: [3240,3240] x: [0.0818389803171158,0.335762768983841] y: [0.197350889444351,0.691627025604248] payload:john kasich>, <Interval3D t: [130860,130860] x: [0.397857904434204,0.620155453681946] y: [0.129200682044029,0.578142821788788] payload:john kasich>, <Interval3D t: [131040,131040] x: [0.408488303422928,0.63443648815155] y: [0.134875744581223,0.592679738998413] payload:john kasich>, <Interval3D t: [131220,131220] x: [0.395539820194244,0.619785845279694] y: [0.134044304490089,0.582561612129211] payload:john kasich>, <Interval3D t: [150120,150120] x: [0.39989972114563,0.629268884658813] y: [0.105802565813065,0.565721154212952] payload:john kasich>, <Interval3D t: [150300,150300] x: [0.685153722763062,0.8379

In [48]:
len(answer.get_allintervals())

19993

In [63]:
answer.get_allintervals()[42341].map_payload(lambda _:None)

[<Interval3D t: [0,806] x: [0.223746836185455,0.827865302562714] y: [0.0735082924365997,0.860915839672089] payload:None>, <Interval3D t: [1012,1418] x: [0.307701647281647,0.850960731506348] y: [0.0436586663126945,0.62831038236618] payload:None>, <Interval3D t: [1564,2039] x: [0.146955773234367,0.816058754920959] y: [0.0312925614416599,0.677523672580719] payload:None>, <Interval3D t: [2374,2684] x: [0.301142513751984,0.828506946563721] y: [0.0507316179573536,0.71288126707077] payload:None>]

In [67]:
answer.get_allintervals()[42341].split(lambda i:i.payload[2])

[<Interval3D t: [59,59] x: [0,1] y: [0,1] payload:(<Interval3D t: [53,59] x: [0.229259967803955,0.630282282829285] y: [0.141163095831871,0.5400550365448] payload:john kasich>, <Interval3D t: [59,59] x: [0.683390200138092,0.790294468402863] y: [0.151130720973015,0.352960616350174] payload:sean hannity>)>, <Interval3D t: [1899,1899] x: [0,1] y: [0,1] payload:(<Interval3D t: [1857,1911] x: [0.208225712180138,0.800987780094147] y: [0.0630991458892822,0.677523672580719] payload:john kasich>, <Interval3D t: [1899,1899] x: [0.63332986831665,0.748618006706238] y: [0.150466561317444,0.36908370256424] payload:sean hannity>)>]

In [69]:
len(vids)

20

In [96]:
keys = answer.get_allintervals().keys()
[(k, answer.get_allintervals()[k].)]

<VideoIntervalCollection3D videos=dict_keys([16403, 84507, 39707, 47207, 44307, 13002, 128907, 301, 45507, 188207, 46707, 42707, 12602, 78107])

In [107]:
def discard(p):
    return None
frame_to_second_collection(answer).get_allintervals()[128907].map_payload(discard)

[<Interval3D t: [3237,3237] x: [0,1] y: [0,1] payload:None>, <Interval3D t: [3240,3240] x: [0,1] y: [0,1] payload:None>, <Interval3D t: [3255,3255] x: [0,1] y: [0,1] payload:None>, <Interval3D t: [3258,3258] x: [0,1] y: [0,1] payload:None>, <Interval3D t: [3273,3273] x: [0,1] y: [0,1] payload:None>, <Interval3D t: [3276,3276] x: [0,1] y: [0,1] payload:None>, <Interval3D t: [3294,3294] x: [0,1] y: [0,1] payload:None>, <Interval3D t: [3333,3333] x: [0,1] y: [0,1] payload:None>, <Interval3D t: [3336,3336] x: [0,1] y: [0,1] payload:None>, <Interval3D t: [3357,3357] x: [0,1] y: [0,1] payload:None>]